In [1]:
from dotenv import load_dotenv
# Load Environment Variables
load_dotenv()

True

In [2]:
# Create an engine to connect to the database

from sqlalchemy.orm import sessionmaker
from sqlalchemy.ext.asyncio import (
    AsyncSession,
    create_async_engine,
)
db_type="sqlite"

aengine = create_async_engine(
    url=f"{db_type}+aiosqlite:///demo_databases/tn_covid_cases_11_may.sqlite"
)

async_session = sessionmaker(
    bind=aengine,
    class_=AsyncSession,
    expire_on_commit=False
)

In [3]:
# Parameters
metric_db_id = "test"
llm = "gpt-4o"
validation_llm = "gpt-4o"
guardrails_llm = "gpt-4o"

sys_message = "Government and health officials in Tamil Nadu, India will ask you questions. You need to help them manage COVID cases and the availablity of beds in health facilities."

db_description = "- bed_vacancies_clinics_11_may: Each row identifies a district and the beds earmarked, occupied and available for COVID cases in the district clinics.\
- bed_vacancies_health_centers_and_district_hospitals_11_may: Each row identifies a district and the beds earmarked, occupied and available, with and without oxygen supply, and with and without ICU support, for COVID cases in the disctrict health centers and hospitals.\
- covid_cases_11_may: Each row identifies a district and the number of people who received treatment, were discharged and died due to COVID.\
"

indicator_vars="district_name" # This should be a comma delimited string in multiple vars


### Single-turn question

In [4]:
import logging

logging.basicConfig(level=logging.INFO)

In [13]:
# Your question
query = {
    "query_text": "How many beds are there in chennai??",
    "query_metadata": {}
}


In [14]:
from askametric.query_processor.query_processor import LLMQueryProcessor

async with async_session() as session:
    qp = LLMQueryProcessor(
        query,
        session,
        metric_db_id,
        db_type,
        llm,
        guardrails_llm,
        sys_message,
        db_description,
        column_description="",
        num_common_values=num_common_values,
        indicator_vars=indicator_vars
    )
    await qp.process_query()
    print(qp.final_answer)

16:25:32 - LiteLLM:INFO: utils.py:2960 - 
LiteLLM completion() model= gpt-4o; provider = openai
INFO:LiteLLM:
LiteLLM completion() model= gpt-4o; provider = openai
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
16:25:33 - LiteLLM:INFO: utils.py:2960 - 
LiteLLM completion() model= gpt-4o; provider = openai
INFO:LiteLLM:
LiteLLM completion() model= gpt-4o; provider = openai
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
16:25:35 - LiteLLM:INFO: utils.py:2960 - 
LiteLLM completion() model= gpt-4o; provider = openai
INFO:LiteLLM:
LiteLLM completion() model= gpt-4o; provider = openai
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
16:25:35 - LiteLLM:INFO: utils.py:2960 - 
LiteLLM completion() model= gpt-4o; provider = openai
INFO:LiteLLM:
LiteLLM completion() model= gpt-4o; provider = openai
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completion

In Chennai, there are a total of 20,334 beds available. This total is derived from two sources: 7,179 beds are available in clinics, and 13,155 beds are available in health centers and district hospitals. This information is based on the latest data collected from these facilities.


### Multi-turn chat

In [10]:
# Simulating a conversation

queries = [{
    "query_text": "How many beds are there in chennai??",
    "query_metadata": {}
},
{
    "query_text": "How about Ranipet??",
    "query_metadata": {}
},
{
    "query_text": "Inge irrukira COVID patients patthi sollu",
    "query_metadata": {}
},
{
    "query_text": "How many beds with oxygen supply in Ariyalur??",
    "query_metadata": {}
},
{
    "query_text": "And how many COVID patients here?",
    "query_metadata": {}
},
{
    "query_text": "Nahin, Ariyalur ke liye batao",
    "query_metadata": {}
},]


In [11]:
from askametric.query_processor.query_processor import MultiTurnQueryProcessor

chat_history = []
async with async_session() as session:
    for query in queries:
        mqp = MultiTurnQueryProcessor(
            query=query,
            asession=session,
            metric_db_id=metric_db_id,
            db_type=db_type,
            llm=llm,
            guardrails_llm=guardrails_llm,
            sys_message=sys_message,
            db_description=db_description,
            column_description="",
            indicator_vars=indicator_vars,
            num_common_values=num_common_values,
            chat_history=chat_history
        )
        await mqp.process_query()
        chat_history.append({"user": mqp.eng_translation["query_text"],
                             "system": mqp.translated_answer})
        print(f"Q: {query['query_text']}")
        print(f"A: {mqp.final_answer}")
        print("\n")

Q: How many beds are there in chennai??
A: In Chennai, there are a total of 20,334 beds available. This total is derived from two sources: 7,179 beds are available in clinics, and 13,155 beds are available in health centers and district hospitals. These numbers represent the sum of earmarked beds, including those with and without oxygen supply, as well as ICU beds.


Q: How about Ranipet??
A: In Ranipet, there are a total of 1,569 beds available. This total is calculated by adding 936 beds from clinics and 633 beds from health centers and district hospitals. This includes beds with and without oxygen supply, as well as ICU beds.


Q: Inge irrukira COVID patients patthi sollu
A: Ranipet-il COVID manithargal patri ketkirirgal. Angu irukkum data padi, Ranipet-il 2018 COVID manithargal irukkirargal. Ivaril 423 per maruthuvamanaiyil irukkirargal, 277 per maruthuvamanaiyil irundhu viduvikkappattirargal, 2 per maranam adainthirargal. Ithu thavira, 2162 per COVID-ai anubavithu maruthuvamanaiyi